In [1]:
import numpy as np
samples = np.load('data/samples_300_pft_ex_norm.npy')
pft = np.load('data/samples_300_pft_ex.npy')

# Add PFT data as a new row to samples array
# First expand pft[4,:] to have the same shape as other rows
pft_row = pft[4, :].reshape(1, -1)  # Shape: (1, n_samples)
samples = np.concatenate((samples, pft_row), axis=0)

print(f"Original samples shape: {np.load('data/samples_300_pft_ex_norm.npy').shape}")
print(f"PFT row shape: {pft_row.shape}")
print(f"Final samples shape: {samples.shape}")
print(f"Sample structure: [ssrd, vpd, gpp, sif, pft]")

Original samples shape: (4, 72000)
PFT row shape: (1, 72000)
Final samples shape: (5, 72000)
Sample structure: [ssrd, vpd, gpp, sif, pft]


In [2]:
# Extract features and target
# Features: SSRD (row 0), VPD (row 1), PFT (row 4)
# Target: GPP (row 2)
X = samples[0:2, :].T  # Shape: (n_samples, 3) - includes PFT
y = samples[2, :]            # Target: GPP

In [3]:
pft_1 = np.zeros(X.shape[0],dtype=X.dtype).reshape(-1,1)
pft_2 = np.zeros(X.shape[0],dtype=X.dtype).reshape(-1,1)
pft_2[:,0] = 1.0
pft_3 = np.zeros(X.shape[0],dtype=X.dtype).reshape(-1,1)

In [4]:
X = np.concatenate((X, pft_1, pft_2, pft_3), axis=1)  # Add PFT columns

In [5]:
# Split data into training and validation sets
# Convert data to PyTorch tensors
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
import torch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)  # Ensure correct shape
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32).unsqueeze(1)

In [6]:
from model import ClimateDataset
from torch.utils.data import DataLoader
# Create DataLoaders
batch_size = 64  # You can adjust this value based on your available memory and GPU capacity
train_loader = DataLoader(ClimateDataset(X_train_tensor, y_train_tensor), batch_size=batch_size, shuffle=True)
val_loader = DataLoader(ClimateDataset(X_val_tensor, y_val_tensor), batch_size=batch_size, shuffle=False)

In [7]:
from model import NeuralNet
from torch import nn, optim
# Initialize model, loss function, and optimizer
input_dim = X.shape[1]  # Number of predictors (ssrd and vpd)
model = NeuralNet(input_dim)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

pretrained_weights_path = "./outputs/model_weights_pre-training_pft_ex.pth"
model.load_state_dict(torch.load(pretrained_weights_path, map_location=device))

# Move model to device
model.to(device)

# Freeze the first two layers
layer_count = 0
for name, param in model.named_parameters():
    if layer_count < 2:  # Freeze first two layers
        param.requires_grad = False
        print(f"🔒 Frozen layer: {name}")
    else:
        param.requires_grad = True
        print(f"🔓 Trainable layer: {name}")
    layer_count += 1

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_train_loss = 0
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)

        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()

    # Validation Step
    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for batch_X, batch_y in val_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            total_val_loss += loss.item()

    avg_train_loss = total_train_loss / len(train_loader)
    avg_val_loss = total_val_loss / len(val_loader)

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.10f}, "
            f"Val Loss: {avg_val_loss:.10f}")


🔒 Frozen layer: model.0.weight
🔒 Frozen layer: model.0.bias
🔓 Trainable layer: model.2.weight
🔓 Trainable layer: model.2.bias
🔓 Trainable layer: model.4.weight
🔓 Trainable layer: model.4.bias
🔓 Trainable layer: model.6.weight
🔓 Trainable layer: model.6.bias
Epoch [1/10], Train Loss: 0.0017276521, Val Loss: 0.0017908210
Epoch [2/10], Train Loss: 0.0016946172, Val Loss: 0.0017108459
Epoch [3/10], Train Loss: 0.0016933567, Val Loss: 0.0017165160
Epoch [4/10], Train Loss: 0.0016976582, Val Loss: 0.0017278078
Epoch [5/10], Train Loss: 0.0016960857, Val Loss: 0.0018123654
Epoch [6/10], Train Loss: 0.0016974373, Val Loss: 0.0017636268
Epoch [7/10], Train Loss: 0.0017062507, Val Loss: 0.0018011674
Epoch [8/10], Train Loss: 0.0016942451, Val Loss: 0.0017976353
Epoch [9/10], Train Loss: 0.0016991862, Val Loss: 0.0017395339
Epoch [10/10], Train Loss: 0.0016937958, Val Loss: 0.0017474511


In [8]:
# Save the model weights
torch.save(model.state_dict(), "./outputs/model_weights_fine-tuning_pft_ex.pth")